In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install kaggle

In [3]:
!mkdir ~/.kaggle
!cp //content/kaggle.json ~/.kaggle/

cp: cannot stat '//content/kaggle.json': No such file or directory


In [4]:
!chmod 600  ~/.kaggle/kaggle.json

chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [5]:
!kaggle datasets download chauri/facebook-hateful-memes

Dataset URL: https://www.kaggle.com/datasets/chauri/facebook-hateful-memes
License(s): unknown
100% 3.93G/3.94G [00:53<00:00, 119MB/s]
100% 3.94G/3.94G [00:53<00:00, 79.2MB/s]


In [6]:
! unzip facebook-hateful-memes.zip

Streaming output truncated to the last 5000 lines.
  inflating: hateful_memes/img/58497.png  
  inflating: hateful_memes/img/58601.png  
  inflating: hateful_memes/img/58602.png  
  inflating: hateful_memes/img/58604.png  
  inflating: hateful_memes/img/58607.png  
  inflating: hateful_memes/img/58609.png  
  inflating: hateful_memes/img/58612.png  
  inflating: hateful_memes/img/58613.png  
  inflating: hateful_memes/img/58621.png  
  inflating: hateful_memes/img/58631.png  
  inflating: hateful_memes/img/58637.png  
  inflating: hateful_memes/img/58640.png  
  inflating: hateful_memes/img/58642.png  
  inflating: hateful_memes/img/58649.png  
  inflating: hateful_memes/img/58672.png  
  inflating: hateful_memes/img/58674.png  
  inflating: hateful_memes/img/58679.png  
  inflating: hateful_memes/img/58690.png  
  inflating: hateful_memes/img/58694.png  
  inflating: hateful_memes/img/58703.png  
  inflating: hateful_memes/img/58706.png  
  inflating: hateful_memes/img/58716.png  
  i

In [7]:
# pip installs
!pip install transformers

In [8]:
!pip install torcheval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.2/179.2 kB 3.6 MB/s eta 0:00:00


In [9]:
%load_ext autoreload
%autoreload 2

# torch
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torcheval.metrics.functional import binary_auroc

# data
import pandas as pd

# model
from transformers import AutoTokenizer, BertForTokenClassification, BertTokenizer,  BertForSequenceClassification, RobertaForSequenceClassification, RobertaTokenizerFast
import torch
from torch.optim import AdamW
from transformers import AutoImageProcessor, ResNetForImageClassification

# general
from PIL import Image
import requests
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tqdm import tqdm

In [10]:
# gpu check
print(torch.__version__)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device=='cuda')

2.2.1+cu121
True


In [11]:
# Hyperparameters
batch_size = 32
lr = 1e-5
epochs = 10

In [12]:
path = "/content/hateful_memes/"

In [13]:
val_seen = pd.read_json(path+'dev_seen.jsonl',lines=True)
val_unseen= pd.read_json(path+'dev_unseen.jsonl',lines=True)
test_seen=pd.read_json(path+'test_seen.jsonl',lines=True)
test_unseen=pd.read_json(path+'test_unseen.jsonl',lines=True)
df_train =pd.read_json(path+'train.jsonl',lines=True)

In [14]:
df_train.label.value_counts()

label
0    5481
1    3019
Name: count, dtype: int64

In [15]:
val_seen.label.value_counts()

label
0    253
1    247
Name: count, dtype: int64

In [16]:
import cv2

class HatefulMemesDataset(Dataset):
    def __init__(self, jsonl_file, root_dir, transform=None):
        self.annotations = pd.read_json(jsonl_file, lines=True)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.annotations)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        id = self.annotations.loc[idx, 'id']
        img_path = os.path.join(self.root_dir, self.annotations.loc[idx, 'img'])
        label = self.annotations.loc[idx, 'label']
        text = self.annotations.loc[idx, 'text']

        try:
            # Load image
            image = cv2.imread(img_path)
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Apply transformations
            if self.transform:
                image = self.transform(image)

            sample = {"id": id, "image": image, "label": label, "text": text}

            return sample

        except Exception as e:
            print(f"Error loading image at index {idx}: {e}")
            return None

In [17]:
hm_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [18]:
import os
train_jsonl = os.path.join(path, "train.jsonl")
val_seen_jsonl = os.path.join(path, "dev_seen.jsonl")
val_unseen_jsonl = os.path.join(path, "dev_unseen.jsonl")
test_seen_jsonl = os.path.join(path, "test_seen.jsonl")
test_unseen_jsonl = os.path.join(path, "test_unseen.jsonl")

In [19]:
# Create datasets
train_dataset = HatefulMemesDataset(jsonl_file=train_jsonl, root_dir=path, transform=hm_transform)
val_seen_dataset = HatefulMemesDataset(jsonl_file=val_seen_jsonl, root_dir=path, transform=hm_transform)
val_unseen_dataset = HatefulMemesDataset(jsonl_file=val_unseen_jsonl, root_dir=path, transform=hm_transform)
test_seen_dataset = HatefulMemesDataset(jsonl_file=test_seen_jsonl, root_dir=path, transform=hm_transform)
test_unseen_dataset = HatefulMemesDataset(jsonl_file=test_unseen_jsonl, root_dir=path, transform=hm_transform)

val_dataset = val_seen_dataset + val_unseen_dataset
test_dataset = test_seen_dataset + test_unseen_dataset

In [20]:
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_unseen_dataset, batch_size=batch_size, shuffle=False)

In [21]:
from torch.utils.data import ConcatDataset

# Combine datasets
val_dataset = ConcatDataset([val_seen_dataset, val_unseen_dataset])
# Combine data loaders
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Combine datasets
test_dataset = ConcatDataset([test_seen_dataset, test_unseen_dataset])
# Combine data loaders
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [22]:
def get_metrics_and_losses(losses, predictions, labels):
  average_loss = losses.mean().item()
  accuracy = (predictions == labels).sum().item() / labels.numel()
  auroc = binary_auroc(predictions, labels)

  return average_loss, accuracy, auroc

In [23]:
def weighted(inputs):
  output = torch.empty(inputs[0].shape).to(device)
  m = nn.ReLU()
  for i in range(inputs[0].shape[0]):
    weights1 = inputs[0][i].to(device)
    weights2 = inputs[1][i].to(device)

    normed1 = weights1/weights1[weights1.argmax(-1)].to(device)
    normed2 = weights2/weights2[weights2.argmax(-1)].to(device)
    output[i] = normed1+normed2
  return output

In [24]:
label2id = {"Hateful":1, "Not Hateful" :0}
id2label = {1: "Hateful", 0: "Not Hateful"}

In [25]:
# model = RoBertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)
model = RobertaForSequenceClassification.from_pretrained("roberta-base", num_labels=2)
modelResNet = ResNetForImageClassification.from_pretrained("microsoft/resnet-152", label2id=label2id, id2label=id2label, ignore_mismatched_sizes=True).to(device)


image_processor = AutoImageProcessor.from_pretrained("microsoft/resnet-152")
# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
tokenizer = RobertaTokenizerFast.from_pretrained("roberta-base")

optimizerRes = AdamW(modelResNet.parameters(), lr=lr)
optimizerRoberta = AdamW(model.parameters(), lr=lr)

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
modelResNet.to(device)
model.to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


config.json:   0%|          | 0.00/69.6k [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(50265, 768, padding_idx=1)
      (position_embeddings): Embedding(514, 768, padding_idx=1)
      (token_type_embeddings): Embedding(1, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-11): 12 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): RobertaSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
             

In [ ]:
criterion = nn.CrossEntropyLoss()
model.train()
modelResNet.train()

for e in range(epochs):

  # training
  train_losses = torch.zeros(len(train_dataloader)).to(device)
  train_predictions = torch.Tensor().to(device)
  train_labels = torch.Tensor().to(device)

  # validation
  validate_losses = torch.zeros(len(val_dataloader)).to(device)
  validate_predictions = torch.Tensor().to(device)
  validate_labels = torch.Tensor().to(device)


  for i, data in tqdm(enumerate(train_dataloader), total=len(train_dataloader), desc="Epoch " + str(e) + " Training"):

    text = data['text']
    image = data["image"].to(device)
    labels = data['label'].to(device).to(torch.int64).to(device)

    optimizerRes.zero_grad()
    optimizerRoberta.zero_grad()

    tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
    scoresBert = model(**tokens)

    with torch.no_grad():
        inputs = image_processor(image, return_tensors="pt", do_rescale = False ).to(device)
    scoresRes = modelResNet(**inputs)


    lossRes = criterion(scoresRes.logits, labels.squeeze(0))
    lossRes.backward()
    optimizerRes.step()

    lossBert = criterion(scoresBert.logits, labels.squeeze(0))
    lossBert.backward()
    optimizerRoberta.step()

    combined_log = weighted((scoresRes.logits, scoresBert.logits))
    # print("Resnet and BERT output", scoresRes.logits, scoresBert.logits)
    predictions = combined_log.argmax(dim=-1)
    # print("final prediction, target", predictions, labels.squeeze(0))

    train_losses[i] = lossRes + lossBert
    train_predictions = torch.cat((train_predictions, predictions))
    train_labels = torch.cat((train_labels, labels))

  train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))
  print(f' Epoch {e}: Training Loss: {train_average_loss}, Training Accuracy: {train_accuracy}, Training AUROC: {train_auroc}')

  with torch.no_grad():
    for i, data in tqdm(enumerate(val_dataloader), total=len(val_dataloader), desc="Validation " + str(e)):

      # get inputs
      text = data['text']
      image = list(data['image'].detach().numpy())
      labels = data['label'].to(device).to(torch.int64)

      tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
      scoresBert = model(**tokens)

      with torch.no_grad():
          inputs = image_processor(image, return_tensors="pt", do_rescale = False ).to(device)
      scoresRes = modelResNet(**inputs)

      combined_log = weighted((scoresRes.logits, scoresBert.logits))
      predictions = combined_log.argmax(dim=-1)

      # loss
      loss = criterion(combined_log, labels)

      # track values for metric logging
      validate_losses[i] = loss
      validate_predictions = torch.cat((validate_predictions, predictions), dim=0)
      validate_labels = torch.cat((validate_labels, labels), dim=0)

    validate_average_loss, validate_accuracy, validate_auroc = get_metrics_and_losses(validate_losses, validate_predictions, validate_labels)
    print(f' Epoch {e}: Validation Loss: {validate_average_loss}, Validation Accuracy: {validate_accuracy}, Validation AUROC: {validate_auroc}')



Epoch 0 Training: 100%|██████████| 266/266 [07:37<00:00,  1.72s/it]


 Epoch 0: Training Loss: 1.2342755794525146, Training Accuracy: 0.648235294117647, Training AUROC: 0.5074066882498539


Validation 0: 100%|██████████| 33/33 [00:42<00:00,  1.29s/it]


 Epoch 0: Validation Loss: 1.3179744482040405, Validation Accuracy: 0.5711538461538461, Validation AUROC: 0.5024955577939495


Epoch 1 Training: 100%|██████████| 266/266 [07:24<00:00,  1.67s/it]


 Epoch 1: Training Loss: 1.1247053146362305, Training Accuracy: 0.6624705882352941, Training AUROC: 0.5288598832704554


Validation 1: 100%|██████████| 33/33 [00:29<00:00,  1.10it/s]


 Epoch 1: Validation Loss: 1.351670742034912, Validation Accuracy: 0.5663461538461538, Validation AUROC: 0.5010336853145007


Epoch 2 Training:  39%|███▊      | 103/266 [02:45<04:20,  1.60s/it]

In [ ]:
train_average_loss, train_accuracy, train_auroc = get_metrics_and_losses(train_losses, train_predictions, train_labels.squeeze(0))
print(train_average_loss, train_accuracy, train_auroc)

In [ ]:
# testing
test_losses = torch.zeros(len(test_dataloader)).to(device)
test_predictions = torch.Tensor().to(device)
test_labels = torch.Tensor().to(device)

with torch.no_grad():
  for i, data in tqdm(enumerate(test_dataloader), total=len(test_dataloader), desc="Testing " + str(e)):

    # get inputs
    text = data['text']
    image = list(data['image'].detach().numpy())
    labels = data['label'].to(device).to(torch.int64)

    tokens = tokenizer(text, return_tensors='pt', padding=True).to(device)
    scoresBert = model(**tokens)

    with torch.no_grad():
        inputs = image_processor(image, return_tensors="pt").to(device)
    scoresRes = modelResNet(**inputs)

    combined_log = weighted((scoresRes.logits, scoresBert.logits))
    predictions = combined_log.argmax(dim=-1)

    # loss
    loss = criterion(combined_log, labels)

    # track values for metric logging
    test_losses[i] = loss
    test_predictions = torch.cat((test_predictions, predictions), dim=0)
    test_labels = torch.cat((test_labels, labels), dim=0)

test_average_loss, test_accuracy, test_auroc = get_metrics_and_losses(test_losses, test_predictions, test_labels)
print(f' Epoch {e}: Test Loss: {test_average_loss}, Test Accuracy: {test_accuracy}, Test AUROC: {test_auroc}')

